# Descripción del modelo

## Modelo Markoviano SEAIHRF en tiempo discreto para el esparcimiento del COVID-19

El espárcimiento de epidemias ha sido exitosamente explicado con modelos matemáticos que consideran las interacciones en una población entre indiviudos suceptibles, infectados y removidos (ver, por ejemplo, [[1]](https://dx.doi.org/10.1017%2FS0950268810002165), [[2]](https://doi.org/10.1111/j.1541-0420.2006.00609.x), [[3]](https://doi.org/10.1038/nature03063) ).

Para nuestras proyecciones seguimos el trabajo de Arenas y sus colaboradores [[4]](https://covid-19-risk.github.io/map/model.pdf), que propusieron una aproximación microscópica Markoviana a tiempo discreto para modelar el esparcimiento de epidemias. En su modelo, cada paso temporal representa un día, y clasifican a una población de $N$ individuos en las siguientes categorías: población susceptible ($S$), expuesta al virus ($E$), asintomática ($A$), infectada ($I$), hospitalizada a UCI ($H$), recuperada ($R$), y fallecida ($F$), donde $S + E + A + I + H + R + F = N$ en todo momento. Adicionalmente, subdividen a la población en 1) el grupo de edad al que pertenecen --específicamente jóvenes (<25), adultos (25-65), y viejos (65+)-- y 2) la región en donde viven, donde cada región está acoplada con las demás usando los patrones de movilidad entre ellas. Presentamos una breve explicación de cada una de las categorías en una lista al final del texto.

Nuestro modelo es una simplificación del de Arenas ya que agregamos a los grupos de edad en una sola categoría y, aunque sí tomamos regiones separadas --en nuestro caso estados de la república--, la dinámica de cada estado es independiente de las demás. Recalcamos que, aunque [nuestro equipo](https://mexicovid19.github.io/Mexico/about.html) está conformado por físicos con experiencia en redes complejas, no somos epidemiólogos, y éste o cualquier otro modelo podría mejorar con la opinión sensible de un experto en el tema.  Tomamos como referencia el modelo de Arenas dado la precisión que tuvo para explicar el espárcimiento del virus en las diferentes regiones autónomas de España.

Para cada estado $i$, considere una población de $N_i$ personas. Definimos la dinámica de esparcimiento de la epidemia en $i$ como

\begin{align}
  \begin{aligned}
    S^i(t+1) &= S^i(t)( 1 - \Pi^i(t) ) \\ 
    E^i(t+1) &= S^i(t) \Pi^i(t) + (1 - \eta) E^i(t) \\
    A^i(t+1) &= \eta E^i(t) + (1 - \alpha) A^i(t) \\
    I^i(t+1) &= \alpha A^i(t) + ( 1 - (\gamma_I + \mu_I + \nu) ) I^i(t) \\
    H^i(t+1) &= \nu I^i(t) + ( 1 - (\gamma_H + \mu_H) ) H^i(t) \\
    R^i(t+1) &= \gamma_I I^i(t) + \gamma_H H^i(t) + R^i(t) \\
    F^i(t+1) &= \mu_I I^i(t) + \mu_H H^i(t) + F^i(t),
  \end{aligned}
  \label{eq:seaihrd}
\end{align}

cuya formulación se explica a continuación. En un día, $t$, la población susceptible se infecta con probabilidad $\Pi^i(t)$, quedando expuesta al virus. Los individuos que quedan expuestos incuban el virus y, en un promedio de $\eta^{-1}$ días, se consideran como asintomáticos que, en un promedio de $\alpha^{-1}$ días, presentan síntomas y se consideran infectados. La diferencia entre un individuo expuesto, uno asintomático y uno infectado es que los primeros no presentan síntomas ni son contagiosos, los segundos tampoco presentan síntomas pero sí son contagiosos y los últimos ya presentan síntomas notables. Los individuos infectados pueden recuperarse naturalmente, a una tasa $\gamma_I$, ir al hospital a una tasa $\nu$, o, en el peor de los casos, fallecer sin ser tratados, a una tasa $\mu_I$. De los pacientes hospitalizados, unos se recuperaran, a una tasa $\gamma_H$, y otros fallecen, a una tasa $\mu_H$. Dichas tasas consideran 1) el tiempo de transición entre una categoría y otra, y 2) la probabilidad de un individuo de cambiar de categoría. 

Cuantificamos la probabilidad de que un individuo quede expuesto al virus de la siguiente manera

\begin{equation}
    \Pi^i(t) = 1 - (1-\beta)^{ \langle k \rangle \left( \frac{ I^i(t) + A^i(t) }{ N^i } \right) }, 
    \label{eq:prob_desease}
\end{equation}

donde $\Pi^i(t)$ depende de la proporción de gente contagiosa en el día $t$. Elaborando mejor, tome $\beta$ como la probabilidad que un individuo suceptible se infecte al ponerse en contacto con uno contagioso (ya sea asintomático o infectado) y tome $\langle k \rangle$ como el número de contactos promedio que alguien tiene durante el día. Asumiendo que la población está bien mezclada y que cada contacto es independiente, el último término en la ecuación (\ref{eq:prob_desease}) nos da la probabilidad de que alguien **no** se contagie el día $t$. 

Ajustamos los parámetros de nuestro modelo con los presentados en [[4]](https://covid-19-risk.github.io/map/model.pdf) y las referencias ahí expuestas. La información sobre la estructura poblacional de México fue tomada de los datos oficiales del INEGI [[5]](https://www.inegi.org.mx/). Para estimar la infectividad $\beta$ en nuestro modelo, tomamos el número básico de reproducción, $R_0$, y notamos que, simplificando el apéndice en [[4]](https://covid-19-risk.github.io/map/model.pdf), se obtiene que $R_0 = \langle k \rangle \log \big( (1-\beta)^{\alpha^{-1} + \eta^{-1}}\big)$. La incertidumbre de nuestro modelo proviene de la incertidumbre de $R_0$. Tomamos el rango de $R_0$ como el intervalo de confianza de $95 \%$ reportado en [[6]](https://www.nejm.org/doi/full/10.1056/NEJMoa2001316), lo que nos permite hacer un cono de proyecciones del número de casos totales.

Con el modelo (\ref{eq:seaihrd}), podemos calcular la evolución nacional del virus sumando para cada estado. Esto es,

\begin{aligned}
  m(t) &= \sum_{i=1}^{32} m^i(t), \\
  m &\in \big\{ S, E, A, I, H, R, D \big\}.
\end{aligned}

Nuestras proyecciones presentan el número total de casos registrados de COVID-19 en México, es decir

$$\text{Casos Totales}(t) = I(t) + H(R) + R(t) + D(t).$$

## Modelo con restricciones de movilidad

La epidemia que estamos viviendo se está esparciendo rápidamente en todo el mundo. Por eso, es importante tomar serias medidas para `#aplanarlacurva` y hacer que haya menos gente infectada al mismo tiempo. En concreto, una medida efectiva se da cuando el gobierno declara contención a nivel nacional, como se estableció por el gobierno mexicano el 23 de marzo de 2020 cuando inició de la [Jornada Nacional de Sana Distancia](https://www.gob.mx/cms/uploads/attachment/file/541687/Jornada_Nacional_de_Sana_Distancia.pdf).

Llamamos $t_c$ al día cuando se hace efectiva dicha contención. A partir de $t_c$, cada individuo disminuye, en promedio, su número de contactos diarios. Sin embargo, estos individuos volverán a sus hogares, donde conviven con la gente que vive ahí. Sea $\kappa_0$ el porcentaje de gente que se queda en casa a partir de la contención, y sea $\sigma$ el número promedio de habitantes por casa. El número promedio de contactos $\langle k \rangle$ disminuye a partir de $t_c$ de la siguiente manera

\begin{equation}
    \langle k \rangle \to \langle k \rangle(t) = (1 - \kappa_0 \Theta( t - t_c ) )\langle k \rangle + \kappa_0(\sigma - 1)\Theta( t - t_c ),
    \label{eq:k_containtment}
\end{equation}

donde $\Theta(x)$ es la función de Heaviside, donde $\Theta(x) = 1$ si $x \geq 0$ y $\Theta(x) = 0$ si $x < 0$. Viendo el lado derecho de la ecuación (\ref{eq:k_containtment}), su primer término representa el número de contactos que un individuo tiene fuera de casa, mientras que el segundo representa los contactos que tiene dentro de ella. 

De las familias que se autoaislan, hay un porcentaje de ellas que no tienen ningún infectado y, por tanto, no serán suceptibles al virus mientras sigan en cuarentena. La cantidad de individuos que viven en el estado $i$ que dejan de ser suceptibles por contención es 

\begin{equation}
    C^i(t_c) = \left( S^i(t_c) + R^i(t_c) \right)^\sigma.
    \label{eq:contained}
\end{equation}

Quitamos a la población definida en (\ref{eq:contained}) de la categoría de susceptibles y los derivamos a una nueva: los individuos contenidos ($C$). Así, podemos redefinir al modelo (\ref{eq:seaihrd}) tomando en cuenta la contención del gobierno. Las ecuaciones resultantes son

\begin{align}
  \begin{aligned}
    S^i(t+1)  &= S^i(t)( 1 - \Pi^i(t) )( 1 - \delta_{t,t_c} \kappa_0 C^i(t_c) ) \\ 
    E^i(t+1)  &= S^i(t) \Pi^i(t) ( 1 - \delta_{t,t_c} \kappa_0 C^i(t_c) ) + (1 - \eta) E^i(t) \\
    C^i(t+1) &= \Theta( t - t_c ) \kappa_0 C^i(t_c) S^i(t_c) \\
    A^i(t+1) &= \eta E^i(t) + (1 - \alpha) A^i(t) \\
    I^i(t+1) &= \alpha A^i(t) + ( 1 - (\gamma_I + \mu_I + \nu) ) I^i(t) \\
    H^i(t+1) &= \nu I^i(t) + ( 1 - (\gamma_H + \mu_H) ) H^(t) \\
    R^i(t+1) &= \gamma_I I^i(t) + \gamma_H H^i(t) + R^i(t) \\
    F^i(t+1) &= \mu_I I^i(t) + \mu_H H^i(t) + F^i(t),
  \end{aligned}
  \label{eq:seaihrd_containtment}
\end{align}

donde $\delta_{t,t_c} = 1$ si $t = t_c$ y  $\delta_{t,t_c} = 0$ si $t \neq t_c$ es la delta de Kronecker.

<hr>

Descripción de cada categoría en la que se divide la población
- **Susceptibles** - Las personas que no han contraído el virus, pero es posible que se contagien al entrar en contacto con algún portador.
- **Expuestos** - Las personas que sí han contraído el virus pero no son contagiosas.
- **Asintomáticos** - Las personas que sí han contraído el virus, sí son contagiosas, pero no presentan síntomas.
- **Infectados** - Las personas que sí presentan síntomas y sí son contagiosas. Lo más probable es que se recuperen pronto en casa.
- **Hospitalizados** - presentan síntomas graves y requieren de cuidados intensivos.
- **Contenidos** - Las personas sanas que se aislaron en casa a partir de la instrucción del gobierno.
- **Recuperados** - Las personas que se recuperan de coronavirus, ya sea en casa o en el hospital.
- **Fallecidos** - Las personas que han fallecido por coronavirus después de estar en cuidados intensivos.

# Referencias

1. Metcalf, C. J. E., Bjørnstad, O. N., Ferrari, M. J., Klepac, P., Bharti, N., **Lopez-Gatell, H.**, & Grenfell, B. T. (2011). The epidemiology of rubella in Mexico: seasonality, stochasticity and regional variation. Epidemiology & Infection, 139(7), 1029-1038.

1. Lekone, P. E., & Finkenstädt, B. F. (2006). Statistical inference in a stochastic epidemic SEIR model with control intervention: Ebola as a case study. Biometrics, 62(4), 1170-1177.

1. Mills, C. E., Robins, J. M., & Lipsitch, M. (2004). Transmissibility of 1918 pandemic influenza. Nature, 432(7019), 904-906.

1. Arenas, Alex, et al. "A mathematical model for the spatiotemporal epidemic spreading of COVID19." medRxiv (2020).

1. Instituto Nacional de Estadística y Geografía (INEGI) https://www.inegi.org.mx/

1. Li, Q., Guan, X., Wu, P., Wang, X., Zhou, L., Tong, Y., ... & Xing, X. (2020). Early transmission dynamics in Wuhan, China, of novel coronavirus–infected pneumonia. New England Journal of Medicine.